In [111]:
import requests
from bs4 import BeautifulSoup, NavigableString, Tag
import csv
import numpy as np

In [162]:
u1 = 'http://www.cballtimeresults.org/performances?division=Overall+Women&page='
page_n = 1
u2 = '&section=10M&sex=W&utf8=%E2%9C%93&year=1999'
url = "http://www.cballtimeresults.org/performances?division=Overall+Women&page={}&amp;section=10M&amp;sex=W&utf8=%E2%9C%93&year={}".format(1, 1999)
#note that &sect normally stands for section char ->§ , escape this by using &amp; to print an actual ampersand
print(url)

http://www.cballtimeresults.org/performances?division=Overall+Women&page=1&amp;section=10M&amp;sex=W&utf8=%E2%9C%93&year=1999


In [9]:
soup_test = BeautifulSoup(requests.get(url).text, 'lxml')
table = soup_test.find("table", attrs={"class": "row-clickable responsive centered rwd-table"})

In [10]:
print(soup_test.title.text)

Credit Union Cherry Blossom Results - performances search


In [14]:
#table finder
table = soup_test.find("table", attrs={"class": "row-clickable responsive centered rwd-table"})

bs4.element.Tag

In [151]:
col_headers = [] #get column headers 
for th in table.find_all("th"):
    #print(th)
    col_headers.append(th.text.replace('\n', ' ').strip())

col_headers    

['Race',
 'Name',
 'Age',
 'Time',
 'Pace',
 'PiS/TiS',
 'Division',
 'PiD/TiD',
 'Hometown']

In [ ]:
for tr in table.find_all("tr")[1]: #table.find_all('tr')[1] is row 1 in table, [0] is the title row, ect
#td is the data in each row? nested under tr
    print(tr)

In [ ]:
soup_test = BeautifulSoup(requests.get(url).text, 'lxml')
table = soup_test.find("table", attrs={"class": "row-clickable responsive centered rwd-table"})

In [146]:

c = 0
r = 9
page_rows = 20
#read all the datapoints as a long array, break into groups of 9 for each row, 20 rows per page
page_holder = np.zeros(shape = (20, 9), dtype=object) #initialize array
for p in range(page_rows):
    test_row = []
    for i in range((0+(c*9)), (9+(c*9))):
        test_row.append(table.find_all("td")[i].text.replace('\n', ''))
    page_holder[c] = test_row
    c = c + 1
    #print(test_row)    

#results in np object array page_holder that has 20 rows, 9 cols for a single page

# Iterate over a single year's paginated results, output to csv

In [207]:
year = 1999
pages_per_year = 118

In [208]:
col_headers = [] #get column headers, create dtype array to serve as vstack anchor
for th in table.find_all("th"):
    #print(th)
    col_headers.append(th.text.replace('\n', ' ').strip())
year_holder = np.array(col_headers, dtype = object)

for n in range(1, pages_per_year):
    u1 = 'http://www.cballtimeresults.org/performances?division=Overall+Women&page='
    #page_n = 1
    u2 = '&section=10M&sex=W&utf8=%E2%9C%93&year=1999'
    url = "http://www.cballtimeresults.org/performances?division=Overall+Women&page={}&amp;section=10M&amp;sex=W&utf8=%E2%9C%93&year={}".format(n, year)
    #print(url)
    soup_test = BeautifulSoup(requests.get(url).text, 'lxml')
    table = soup_test.find("table", attrs={"class": "row-clickable responsive centered rwd-table"})
    c = 0
    r = 9
    page_rows = 20
    #read all the datapoints as a long array, break into groups of 9 for each row, 20 rows per page
    page_holder = np.zeros(shape = (20, 9), dtype=object) #initialize array
    for p in range(page_rows):
        test_row = []
        for i in range((0+(c*9)), (9+(c*9))):
            test_row.append(table.find_all("td")[i].text.replace('\n', ''))
        page_holder[c] = test_row
        c = c + 1
    #print("page {} added".format(n))
    year_holder = np.vstack((year_holder, page_holder)) #add each page onto the bottom of the column headers

    

ok works for 1999, much cleaner than manual XML space parsing

manual inputs for this process are inputing the number of pages, and naming the csv output


In [240]:
#function version
def year_scraper(year, pages_per_year):
    #get column headers from start point of scrape
    url = "http://www.cballtimeresults.org/performances?division=Overall+Women&page={}&amp;section=10M&amp;sex=W&utf8=%E2%9C%93&year={}".format(1, year)
    soup_test = BeautifulSoup(requests.get(url).text, 'lxml')
    table = soup_test.find("table", attrs={"class": "row-clickable responsive centered rwd-table"})

    col_headers = [] #get column headers, create dtype array to serve as vstack anchor
    for th in table.find_all("th"):
    #print(th)
        col_headers.append(th.text.replace('\n', ' ').strip())
    year_holder = np.array(col_headers, dtype = object)

    for n in range(1, pages_per_year):
        url = "http://www.cballtimeresults.org/performances?division=Overall+Women&page={}&amp;section=10M&amp;sex=W&utf8=%E2%9C%93&year={}".format(n, year)
    #print(url)
        soup_test = BeautifulSoup(requests.get(url).text, 'lxml')
        table = soup_test.find("table", attrs={"class": "row-clickable responsive centered rwd-table"})
        c = 0
        r = 9
        page_rows = 20
    #read all the datapoints as a long array, break into groups of 9 for each row, 20 rows per page
        page_holder = np.zeros(shape = (20, 9), dtype=object) #initialize array

        for p in range(page_rows):
            test_row = []
            for i in range((0+(c*9)), (9+(c*9))):
                test_row.append(table.find_all("td")[i].text.replace('\n', ''))
            page_holder[c] = test_row
            c = c + 1
    #print("page {} added".format(n))
        year_holder = np.vstack((year_holder, page_holder)) #add each page onto the bottom of the column headers


In [237]:
#out of index exception testing
url = "http://www.cballtimeresults.org/performances?division=Overall+Women&page={}&amp;section=10M&amp;sex=W&utf8=%E2%9C%93&year={}".format(119, 1999)
soup_test = BeautifulSoup(requests.get(url).text, 'lxml')
table = soup_test.find("table", attrs={"class": "row-clickable responsive centered rwd-table"})

(table.find_all("td")) == []

True

In [239]:
year_scraper(1999, 118) #could write an exception if the window
np.savetxt('soup.1999.csv', year_holder, delimiter = ',', fmt = "%s") #%s as fmt option should save everything as a string

IndexError: list index out of range

In [ ]:
year_scraper(2001, 118)
np.savetxt('soup.2001.csv', year_holder, delimiter = ',', fmt = "%s") #%s as fmt option should save everything as a string

In [ ]:
year_scraper(2002, 118)
np.savetxt('soup.2002.csv', year_holder, delimiter = ',', fmt = "%s") #%s as fmt option should save everything as a string

In [ ]:
year_scraper(2003, 118)
np.savetxt('soup.2003.csv', year_holder, delimiter = ',', fmt = "%s") #%s as fmt option should save everything as a string

In [ ]:
year_scraper(2004, 118)
np.savetxt('soup.2004.csv', year_holder, delimiter = ',', fmt = "%s") #%s as fmt option should save everything as a string

In [ ]:
year_scraper(2005, 118)
np.savetxt('soup.2005.csv', year_holder, delimiter = ',', fmt = "%s") #%s as fmt option should save everything as a string

In [ ]:
year_scraper(2006, 118)
np.savetxt('soup.2006.csv', year_holder, delimiter = ',', fmt = "%s") #%s as fmt option should save everything as a string

In [ ]:
year_scraper(2007, 118)
np.savetxt('soup.2007.csv', year_holder, delimiter = ',', fmt = "%s") #%s as fmt option should save everything as a string

In [ ]:
year_scraper(2008, 118)
np.savetxt('soup.2008.csv', year_holder, delimiter = ',', fmt = "%s") #%s as fmt option should save everything as a string

In [ ]:
year_scraper(2009, 118)
np.savetxt('soup.2009.csv', year_holder, delimiter = ',', fmt = "%s") #%s as fmt option should save everything as a string

In [ ]:
year_scraper(2010, 118)
np.savetxt('soup.2010.csv', year_holder, delimiter = ',', fmt = "%s") #%s as fmt option should save everything as a string